In [11]:
import folium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time
import os
import requests

In [2]:
def download_satellite_image(lat, lon, zoom=13, size=(512, 512), save_path='satellite_image.png'):
    # Use public Sentinel-2 WMS layer (from Sentinel Hub)
    tile_url = (
        f"https://services.sentinel-hub.com/ogc/wms/"
        f"demo?REQUEST=GetMap&SERVICE=WMS&VERSION=1.3.0&"
        f"BBOX={lat-0.02},{lon-0.02},{lat+0.02},{lon+0.02}&"
        f"CRS=EPSG:4326&WIDTH={size[0]}&HEIGHT={size[1]}&"
        f"LAYERS=TRUE_COLOR&FORMAT=image/png&TRANSPARENT=FALSE&"
        f"TIME=2023-07-01/2023-07-10"  # Date range for recent imagery
    )

    print(f"Fetching satellite image for lat={lat}, lon={lon}")
    response = requests.get(tile_url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Image saved to: {save_path}")
    else:
        print("Failed to download image. Status code:", response.status_code)

# Example usage
latitude = 29.38
longitude = 79.46
download_satellite_image(latitude, longitude)


Fetching satellite image for lat=29.38, lon=79.46
Failed to download image. Status code: 404


In [3]:
def visualize_on_map(lat, lon):
    m = folium.Map(location=[lat, lon], zoom_start=13)
    folium.TileLayer(
        tiles="https://services.sentinel-hub.com/ogc/wms/demo"
              "?REQUEST=GetMap&SERVICE=WMS&VERSION=1.3.0&"
              "LAYERS=TRUE_COLOR&FORMAT=image/png&"
              "TIME=2023-07-01/2023-07-10",
        attr='Sentinel-Hub',
        name='Sentinel-2',
        overlay=True,
        control=True,
    ).add_to(m)
    folium.Marker([lat, lon], tooltip="Center").add_to(m)
    return m

# View in notebook or save as HTML
visualize_on_map(latitude, longitude).save("map.html")


In [13]:
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=800,800")

# Optional: if Chrome is installed in a custom path, set binary location
options.binary_location = "/usr/bin/google-chrome-stable"  # Uncomment and set if needed

# ✅ Correct way to initialize driver
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


def km_to_deg(km):
    # Approximation: 1 degree ≈ 111 km
    return km / 111.0

def generate_satellite_map(lat, lon, radius_km, zoom=14, output_prefix="map_output"):
    delta = km_to_deg(radius_km)

    # Create folium map
    m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles=None)
    
    # Add Esri Satellite tile
    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr="Esri",
        name="Esri Satellite",
        overlay=False,
        control=True
    ).add_to(m)

    # Draw boundary (bounding box)
    bounds = [[lat - delta, lon - delta], [lat + delta, lon + delta]]
    folium.Rectangle(bounds=bounds, color='red', fill=False).add_to(m)
    folium.Marker([lat, lon], tooltip="Center").add_to(m)

    # Save as HTML
    html_file = f"{output_prefix}.html"
    m.save(html_file)
    print(f"[✔] HTML map saved to: {html_file}")

    # Render and save as image using Selenium
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=800,800")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        driver.get(f"file://{os.path.abspath(html_file)}")
        time.sleep(3)  # wait for map to render
        
        screenshot_path = f"{output_prefix}.png"
        driver.save_screenshot(screenshot_path)
        print(f"[✔] PNG image saved to: {screenshot_path}")
    finally:
        driver.quit()  # Ensure driver is always closed

# 🔽 Example usage:
generate_satellite_map(
    lat=11.039168,
    lon=77.069411,
    radius_km=1  # You can take this input from user
)


[✔] HTML map saved to: map_output.html
[✔] PNG image saved to: map_output.png
[✔] PNG image saved to: map_output.png


In [ ]:
import geemap
import ee
import numpy as np

# Initialize Earth Engine
ee.Initialize()

def extract_landcover_matrix(lat, lon, radius_km):
    # Define region
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(radius_km * 1000).bounds()

    # Use ESA WorldCover 2021 dataset (Sentinel-based, 10m resolution)
    landcover = ee.Image("ESA/WorldCover/v100/2020")

    # Clip to region
    clipped = landcover.clip(region)

    # Export as NumPy array (low-res thumbnail for prototyping)
    url = clipped.getThumbURL({
        'region': region,
        'dimensions': 256,
        'format': 'png',
        'min': 10,
        'max': 100
    })

    print(f"[✔] Preview your classified land cover image: {url}")

    return url


: 